In [ ]:
# This project focuses on brain CT image segmentation and denoising using Deep Learning.
# Al Smith and Will Newman
# c 2024

# Importing the necessary libraries:

In [1]:
# Tasks:
# 1) Load data from CQ500
# 2) Preprocess the data (noise addition, downsampling, agumentation, etc)
# 3) Build a 2D U-Net model for segmentation
# 4) Denoising Model for noisy CT images
# 5) Train the model
# 6) Evaluate the model